# Split Amazon

This splits the Amazon data for a train-test sweep.

In [ ]:
library(readr)
library(dplyr)

In [ ]:
options(repr.matrix.max.rows = 20)

## Explicit Ratings

In [ ]:
ratings = read_csv("build/az-ratings.csv", col_names = TRUE, col_types='iid') %>%
    rename(user=userID, item=bookID)

In [ ]:
dim(ratings)

How many unique users?

In [ ]:
ratings %>%
    select(user) %>%
    distinct() %>%
    nrow()

In [ ]:
ratings %>%
    group_by(user) %>%
    summarize(nratings=n()) %>%
    filter(nratings >= 10) %>%
    ungroup() %>%
    nrow()

In [ ]:
explicit.test.users = ratings %>%
    group_by(user) %>%
    summarize(nratings=n()) %>%
    filter(nratings >= 10) %>%
    sample_n(5000)
explicit.test.users
dim(explicit.test.users)

In [ ]:
ratings.group = ratings %>%
    group_by(user) %>%
    mutate(urid = sample(n())) %>%
    ungroup()

In [ ]:
test.ratings = explicit.test.users %>%
    select(user) %>%
    inner_join(ratings.group) %>%
    filter(urid <= 5) %>%
    select(user, item, rating)
dim(test.ratings)

In [ ]:
train.ratings = explicit.test.users %>%
    right_join(ratings.group) %>%
    filter(urid > 5 | is.na(nratings)) %>%
    select(user, item, rating)
dim(train.ratings)

In [ ]:
write_csv(test.ratings, "build/az-ratings-test.csv")
write_csv(train.ratings, "build/az-ratings-train.csv")